In [3]:
pip install mlflow pycaret



     ---------------------------------------- 10.4/10.4 MB 8.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.1
    Uninstalling pandas-2.2.1:
      Successfully uninstalled pandas-2.2.1


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Admin\\anaconda3\\Lib\\site-packages\\~andas.libs\\msvcp140-ef6047a69b174ada5cb2eff1d2bc9a62.dll'
Consider using the `--user` option or check the permissions.



In [4]:
from mlflow import *
import pandas as pd
import numpy as np
import pycaret
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sklearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from wordcloud import WordCloud, STOPWORDS
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.callbacks import EarlyStopping
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import warnings

warnings.filterwarnings("ignore")
pd.options.display.max_columns=None
pd.options.display.max_rows=None
pd.options.display.max_colwidth=None
%matplotlib inline

In [5]:
dff=pd.read_csv("./Annotations_Metadata.csv")
dff.head()

,file_id,user_id,subforum_id,num_contexts,label
0,12834217_1,572066,1346,0,noHate
1,12834217_2,572066,1346,0,noHate
2,12834217_3,572066,1346,0,noHate
3,12834217_4,572066,1346,0,hate
4,12834217_5,572066,1346,0,noHate


In [6]:

with open('Text file/12834217_1.txt', 'r') as file:
    #Read the entire contents of the file
    content = file.read()
    print(content)


As of March 13th , 2014 , the booklet had been downloaded over 18,300 times and counting .


In [7]:
def store_file(file_id):
    with open(f"Text file/{file_id}.txt",'r', encoding='utf-8') as f:
        content=f.read()
        return content


In [8]:
dff['text']=dff['file_id'].apply(store_file)

In [9]:
dff.to_csv("hate speech dataset.csv",index=False)

In [10]:
dff=pd.read_csv("/content/sample_data/hate speech dataset.csv")
dff.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/hate speech dataset.csv'

In [ ]:
dff['user_id'].value_counts()

In [ ]:
dff['num_contexts'].value_counts()

In [ ]:
df=dff[['text','label']]

In [ ]:
df.head()

In [ ]:
df.duplicated().sum()

In [ ]:
duplicate_rows = df[df.duplicated()]
duplicate_rows

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

In [ ]:
filtered_df = df[df['label'].isin(['hate', 'noHate'])]

In [ ]:
filtered_df['label'].value_counts()

In [ ]:
df=filtered_df.copy()

In [ ]:
df['label'].value_counts()

In [ ]:
df['label'] = df['label'].map({'noHate': 0, 'hate': 1})

#  Data Overview

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df.nunique()

#  Class Distribution

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(x='label',data=df)
plt.xlabel("Hate Speech Label")
plt.ylabel('Count')
plt.title("Class Distribution")
plt.show()

# Using Word Cloud

In [ ]:
text=' '.join(df['text'].astype(str))
wordcloud=WordCloud(width=800, height=500,background_color='white').generate(text)
plt.figure(figsize=(10,8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud')
plt.show()

# Length Distribution

In [ ]:
df['text_length']=df['text'].apply(lambda x: len(str(x)))
plt.figure(figsize=(10,6))
sns.histplot(df['text_length'],kde=True)
plt.xlabel('Text Length')
plt.ylabel('Frequency')
plt.title('Length Distribution')
plt.show()

In [ ]:
dff = dff[dff['label'].isin(['hate', 'noHate'])]

In [ ]:
user_data = dff.groupby('user_id')['label'].sum().reset_index()



In [ ]:
# Plot the distribution of hate speech counts per user
plt.figure(figsize=(75, 75))
sns.histplot(x='label', data=user_data, bins=20)
plt.xlabel('Hate Speech Count')
plt.ylabel('Number of Users')
plt.title('Distribution of Hate Speech Counts per User')
plt.show()

In [ ]:
df['clean_text']=df['text'].str.lower()
df['clean_text']=df['clean_text'].str.replace('[^a-zA-Z0-9\s]','')

In [ ]:
df.head()

In [ ]:
vectorizer=TfidfVectorizer(max_features=5000)
X=vectorizer.fit_transform(df['clean_text']).toarray()
y=df['label'].values


In [ ]:
X_train,X_test, y_train,y_test=train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
over=RandomOverSampler(sampling_strategy='minority')
under=RandomUnderSampler(sampling_strategy='majority')

steps=[('o',over),('u',under)]

pipeline=Pipeline(steps=steps)

In [ ]:
X_train_resampled, y_train_resampled=pipeline.fit_resample(X_train,y_train)

In [ ]:
lr=LogisticRegression()
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
gs_lr=GridSearchCV(estimator=lr, param_grid=param_grid,cv=10,scoring='accuracy')
gs_lr.fit(X_train_resampled, y_train_resampled)

In [ ]:
print("Best Estimator:", gs_lr.best_estimator_)
print("Best Params:", gs_lr.best_params_)
print("Best Score:",gs_lr.best_score_)

In [ ]:
y_pred_lr=gs_lr.predict(X_test)

In [ ]:
accuracy=accuracy_score(y_test, y_pred_lr)
print("Test Accuracy:",accuracy)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=1)
param_grid={'criterion':["gini", "entropy", "log_loss"],
            'splitter':["best", "random"]}

gs_dt=GridSearchCV(estimator=dt, param_grid=param_grid, cv=10, scoring='accuracy')
gs_dt.fit(X_train_resampled, y_train_resampled)

In [ ]:
print("Best Estimator:", gs_dt.best_estimator_)
print("Best Params:", gs_dt.best_params_)
print("Best Score:",gs_dt.best_score_)

In [ ]:
y_pred_dt=gs_dt.predict(X_test)

In [ ]:
accuracy_dt=accuracy_score(y_pred_dt, y_test)

In [ ]:
"accuracy",accuracy_dt

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
param_grid={'n_estimators':[10,25,50,75,100]}

gs_rf=GridSearchCV(estimator=rf, param_grid=param_grid, cv=10, scoring='accuracy')
gs_rf.fit(X_train_resampled, y_train_resampled)
#rf.fit(X_train_resampled, y_train_resampled)

In [ ]:
print("Best Estimator:", gs_rf.best_estimator_)
print("Best Params:", gs_rf.best_params_)
print("Best Score:",gs_rf.best_score_)

In [ ]:
#y_pred_rf=gs_rf.predict(X_test)

In [ ]:
#accuracy_rff=accuracy_score(y_pred_rff, y_test)
#accuracy_rff

In [ ]:
y_pred_rf=gs_rf.predict(X_test)

In [ ]:
accuracy_rf=accuracy_score(y_pred_rf, y_test)
accuracy_rf

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
param_grid={'n_neighbors':[1,5,10,25,50],
            'weights':['uniform','distance']
}
gs_knn=GridSearchCV(estimator=knn, param_grid=param_grid, cv=10, scoring='accuracy')
gs_knn.fit(X_train_resampled, y_train_resampled)

In [ ]:
y_pred_knn=gs_knn.predict(X_test)

In [ ]:
accuracy_knn=accuracy_score(y_pred_knn,y_test)
accuracy_knn

In [ ]:
model=Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=X.shape[1]))
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3)


In [ ]:
model.fit(X_train_resampled, y_train_resampled,
          epochs=25, batch_size=32,
          validation_split=0.1, steps_per_epoch=10,
          callbacks=[early_stopping])

In [ ]:
y_pred = model.predict(X_test)


In [ ]:
y_pred = (y_pred > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [ ]:
accuracy

In [ ]:
len(y_test)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
metric=confusion_matrix(y_test, y_pred)

In [ ]:
ConfusionMatrixDisplay(metric)
plt.show()

In [ ]:

false_positives = (y_test == 0) & (y_pred == 1)
false_negatives = (y_test == 1) & (y_pred == 0)

print("False Positive Analysis:")
print(false_positives)

print("False Negative Analysis:")
print(false_negatives)

In [ ]:
df_test=pd.DataFrame()

In [ ]:
df_test = pd.DataFrame({'text': ["Click on the `` DOWNLOAD ( 7.42 MB ) '' green banner link ."]})

In [ ]:
print(df_test)

In [ ]:
df_test['clean_text']=df_test['text'].str.lower()
df_test['clean_text']=df_test['clean_text'].str.replace('[^a-zA-Z0-9\s]','')

In [ ]:
df_test.head()

In [ ]:
raw=vectorizer.fit_transform(df_test['clean_text']).toarray()

In [ ]:
result=model.predict(raw)

In [ ]:
result = (result > 0.5).astype(int)

In [ ]:
result

In [ ]:
if result[0][0]==0:
  print("No Hate")
else:
  print("Hate")